<a href="https://colab.research.google.com/github/manularrea/Deep-Learning/blob/main/hungerGames_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica: Los Juegos del Hambre

ESTUDIANTE: `MANUELA LARREA, JERSON PEÑA`

<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/breakfast.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/hamburger.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fruits.jpg" style="width:300px;height:300px;"></td>
</tr></table>

En esta práctica vamos a enfrentarnos a un problema desafiante de clasificación de imágenes, construyendo una red neuronal profunda que sea capaz de clasificar entre diferentes tipos de comida. ¡Que comiencen los Juegos del Hambre!

## Guidelines

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema, y a conseguir una calificación más alta. ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda hacer uso [Google Colaboratory](https://colab.research.google.com/). Asegúrate de [conectar una GPU](https://colab.research.google.com/notebooks/gpu.ipynb), y de haber [deactivado otras sesiones que tuvieras activas](https://stackoverflow.com/a/53441194/2436578).

El siguiente código mostrará todas las gráficas en el propio notebook en lugar de generar una nueva ventana.

In [40]:
import matplotlib.pyplot as plt
%matplotlib inline

## Obtención de datos

Vamos a usar un dataset de imágenes de comida disponible en [Kaggle](https://www.kaggle.com/trolukovich/food11-image-dataset).

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Usa la celda inferior para descargar el dataset de Kaggle. ¡Ojo! Cuando entregues este notebook no dejes en él tus credenciales, ya que son personales a tu usuario de Kaggle.
</font>

***

In [41]:
####### INSERT YOUR CODE HERE
import os

os.environ["KAGGLE_USERNAME"] = "albarji"
os.environ["KAGGLE_KEY"] = "c98bec07523d01caaa7f117668cf22ce"

!kaggle datasets download trolukovich/food11-image-dataset --unzip


Dataset URL: https://www.kaggle.com/datasets/trolukovich/food11-image-dataset
License(s): CC0-1.0
 98% 1.06G/1.08G [00:11<00:00, 209MB/s]
100% 1.08G/1.08G [00:11<00:00, 104MB/s]


Revisa ahora la carpeta en la que has descargado los datos. Verás que contiene 3 subdirectorios:

* **training**, contiene las imágenes a utilizar para entrenar el modelo.
* **validation**, contiene imágenes adicionales que podrías usar como datos de entrenamiento adicionales, o para algún tipo de estrategia de validación como Early Stopping.
* **evaluation**, contiene las imágenes que debes utilizar para testear el modelo. Las imágenes de esta carpeta **solo** pueden utilizarse para medir el rendimiento del modelo tras su entrenamiento, y para nada más.

Además de esto, dentro de cada una de estas carpetas encontrarás una subcarpeta para cada una de las 11 clases de comida:

* Bread (panes)
* Dairy product (lácteos)
* Dessert (postres)
* Egg (huevos)
* Fried food (fritos)
* Meat (carnes)
* Noodles-Pasta (pasta)
* Rice (arroz)
* Seafood (pescado y marisco)
* Soup (sopas)
* Vegetable-Fruit (vegetales y frutas)

Para facilitar los pasos de procesamiento que vendrán a continuación, vamos a definir algunas variables que nos indiquen dónde están almacenados los diferentes conjuntos de datos.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea variables <b>TRAINDIR</b>, <b>VALDIR</b> y <b>TESTDIR</b>, cada una conteniendo una cadena de texto con la ruta al directorio donde están los datos de entrenamiento, validación y evaluación, respectivamente.
</font>

***

In [42]:
####### INSERT YOUR CODE HERE
TRAINDIR, VALDIR, TESTDIR = 'training','validation', 'evaluation'

### Reducción de clases

Con el fin de hacer este problema más accesible de cara a la práctica, vamos a centrarnos solo en seis de las clases de comida disponibles: `Bread`, `Dairy product`, `Dessert`, `Egg`, `Fried food` y `Meat`. Para ello, se provee el código siguiente, que elimina de los datos descargados las carpetas correspondientes a imágenes de las otras clases.

In [43]:
from glob import glob
import os

valid_classes = {"Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat"}
datasets = {TRAINDIR, VALDIR, TESTDIR}

for dataset in datasets:
    for classdir in glob(f"{dataset}/*"):  # Find subfolders with classes
        if classdir.split("/")[-1] not in valid_classes:  # Ignore those in valid_classes
            print(f"Deleting {classdir}...")
            for fname in glob(f"{classdir}/*.jpg"):  # Remove each image file
                os.remove(fname)
            os.rmdir(classdir)  # Remove folder

Deleting training/Soup...
Deleting training/Noodles-Pasta...
Deleting training/Rice...
Deleting training/Vegetable-Fruit...
Deleting training/Seafood...
Deleting validation/Soup...
Deleting validation/Noodles-Pasta...
Deleting validation/Rice...
Deleting validation/Vegetable-Fruit...
Deleting validation/Seafood...
Deleting evaluation/Soup...
Deleting evaluation/Noodles-Pasta...
Deleting evaluation/Rice...
Deleting evaluation/Vegetable-Fruit...
Deleting evaluation/Seafood...


### Carga de datos

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Utiliza la función <i>image_dataset_from_directory</i> para crear tres objetos de tipo Dataset, uno para cada una de las particiones de datos (training, validation, evaluation)
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
¿Cuál es el tamaño ideal al que cargar las imágenes en memoria?
Esto es algo con lo que tendrás que experimentar a lo largo del notebook, por lo que probablemente tendrás que volver más de una vez a esta celda y probar cambiando ese tamaño.
</font>

***

Lo más flexible aquí es tener una función lo suficientemente modular para poder experimentar con el tamaño de las imagenes (image_size)y el tamaño del lote (batch_size).

Iniciamos con un experimento que toma tamaños de imagen de 32x32 y 64x64. Estos tamaños son los suficientemente pequeños para tener un procesamiento "respetable", y lo suficientemente grandes como para poder capturar características importantes de las imágenes.

In [44]:
####### INSERT YOUR CODE HERE
from tensorflow.keras.preprocessing import image_dataset_from_directory

def create_datasets(traindir = TRAINDIR, valdir = VALDIR, testdir = TESTDIR ,
                    image_size = 32,
                    batch_size = 64,):
  label_mode = 'categorical'

  train_dataset = image_dataset_from_directory(traindir,
                                               image_size=(image_size,image_size),
                                               batch_size=batch_size,
                                               label_mode=label_mode
                                               )

  val_dataset = image_dataset_from_directory(valdir,
                                               image_size=(image_size,image_size),
                                               batch_size=batch_size,
                                               label_mode=label_mode
                                               )

  test_dataset = image_dataset_from_directory(testdir,
                                               image_size=(image_size,image_size),
                                               batch_size=batch_size,
                                               label_mode=label_mode
                                               )

  return train_dataset, val_dataset, test_dataset

In [45]:
train_dataset, val_dataset, test_dataset = create_datasets()

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


# REESCALAR???

## Construyendo tu propia red

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Diseña una red neuronal profunda que obtenga el mejor acierto posible sobre los datos de test. Puedes usar los datos de entrenamiento y validación como te parezca mejor, pero <b>sólo</b> puedes usar los datos de test para evaluar el acierto de tu modelo. Debes obtener una red capaz de alcanzar al menos un 45% de acierto sobre los datos de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunas recomendaciones y estrategias que pueden ayudar a mejorar tu diseño de red:
    
- <b>Arquitectura</b>: usa todos los trucos que has aprendido en los ejercicios anteriores: capas convolucionales + pooling, activaciones ReLU, dropout... asegúrate también de utilizar un buen optimizador, con una función de error (loss) adecuada, así como una función de activación en la capa de salida que sea adecuada para esta clase de problema (clasificación multiclase).
- <b>Desarrollo incremental</b>: empieza por redes pequeñas, con un número pequeño de parámetros, de forma que puedas comprobar rápidamente qué tal funcionan. Después, puedes hacer tu red más grande en tres direcciones: mayor tamaño de imágenes de entrada, más capas, y más kernels por capa convolucional o unidades por capa densa. Si aumentas el tamaño de las imágenes de entrada, asegúrate de añadir también más capas Convolution+Pooling, para que así a la capa Flatten solo lleguen imágenes muy pequeñas (10x10 píxeles o menos).
- <b>Tamaño de imágenes</b>: configurar los `Dataset` para que carguen imágenes de mayor tamaño puede mejorar significativamente el rendimiento de tu red. Pero ten cuidado, ¡también puedes encontrarte errores de falta de memoria (CUDA memory error) si cargas imágenes a un tamaño demasiado grande! Para esta práctica, un tamaño mayor a 256 puede ser demasiado grande...
- <b>Controlar el número de épocas</b>: Usa una <a href="https://keras.io/api/callbacks/early_stopping/">**estrategia de EarlyStopping**</a> para monitorizar el loss de los datos de validación, y así detener el entrenamiento cuando tras un número de épocas esa loss no haya decrecido. Configurar la EarlyStopping para restaurar los mejores parámetros encontrados durante la optimización también puede resultarte útil.
- <b>Sobreajuste</b>: si tu red obtiene un accuracy casi perfecto en entrenamiento, puede que estés sufriendo sobreajuste (aunque puede que no...). Prueba a incrementar el nivel de Dropout en las capas Dense para comprobar si así obtienes mejoras en el test.
- <b>Demasiado bueno para ser verdad</b>: si tu red obtiene resultados muy buenos, del orden del 90% o más de acierto en test... sospecha. Es posible que estés mezclando los datos de entrenamiento y test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
Como ejercicio avanzado, añade las siguiente estrategias a tu red:

- Usa **técnicas de "image augmentation"** para aumentar artificialmente tu dataset de entrenamiento. Para ello, explora las <a href="https://keras.io/api/layers/preprocessing_layers/image_augmentation/">capas de augmentation disponibles en Keras</a>.
- Usa capas de <a href="https://keras.io/api/layers/normalization_layers/batch_normalization/">BatchNormalization</a> para facilitar el entrenamiento de la red. Revisa en las diapositivas de clase cuál es la forma adecuada de colocarlas en la red.
    
Usando estos trucos y los mencionados en el punto anterior, es posible obtener más de un 60% de acierto en el conjunto de test.

</font>

***

## Definición de la arquitectura:
Vamos a seguir un diseño típico de red neuronal convolucional para problemas de clasificación de imagenes.

Este diseño comienza con capas convolucionales y de agrupación para extraer características de las imagenes, luego tiene capas densas para clasificar esas características en las clases de interés.

Usamos activaciones relu en las capas convolucionales y densas para introducir no linealidades y dorpout para regularización (para reducir sobreajuste)

In [46]:
####### INSERT YOUR CODE HERE
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

def build_model(input_shape, num_classes):
  model= Sequential()

  # Capas convolucionales y de agrupación
  model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D((2,2)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))

  # Capa de aplanamiento y capas densas
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))

  # Capa de salida
  model.add(Dense(num_classes, activation='softmax'))

  return model

In [47]:
image_size = (64,64)
num_classes = 6

## Compilacion
Usamos el famoso y muy usado optimizador Adam para entrenar redes neuronales.
Como estamos haciendo una clasificación multiclase, establecemos la función de pérdida en categorical_crossentropy y monitoreamos la precisión con la métrica accuracy.

In [48]:
model = build_model(input_shape=(image_size[0], image_size[1], 3),
                    num_classes=num_classes)

## Entrenamiento con aumentación de imágenes
Queremos introducir variaciones en los datos de entrenamiento (rotaciones, desplazamientos, zooms) para que el modelo se vuelva mas robusto a estas variaciones.

In [49]:
from keras.preprocessing.image import ImageDataGenerator

# Generador de aumentación de imágenes
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [50]:
import numpy as np
from keras.callbacks import EarlyStopping

# Iterar manualmente sobre train_dataset para obtener matrices NumPy de datos y etiquetas
train_images = []
train_labels = []

for images, labels in train_dataset:
    train_images.append(images.numpy())
    train_labels.append(labels.numpy())

train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)


In [51]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [52]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
image_size = (64,64)
batch_size = 32

test_dataset = image_dataset_from_directory(
    TESTDIR,
    image_size = (image_size[0], image_size[1]),
    batch_size = batch_size,
    label_mode = 'categorical'
)

Found 2070 files belonging to 6 classes.


In [54]:
# Entrenar el modelo con aumentación de imágenes
history=model.fit(
    datagen.flow(train_images, train_labels),
    epochs=50,
    validation_data=val_dataset,
    callbacks=[early_stopping]
)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node sequential_2/dense_6/MatMul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/p

## Evaluación
Con el set de test se evalua el rendimiento del modelo... vamos a ver qué tal es la capacidad con datos no vistos

In [55]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Accuracy en el set de test: {test_accuracy * 100:.2f}%')

65/65 [==============================] - 3s 38ms/step - loss: 22.6188 - accuracy: 0.0952
Accuracy en el set de test: 9.52%


## Red con transferencia del aprendizaje

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Usando la estrategia "bottleneck" implementa una red que haga transferencia del aprendizaje desde la red VGG16. Si lo haces correctamente, esta red debe obtener mejores resultados que la red que diseñaste en el apartado anterior, con al menos un 80% de acierto sobre el conjunto de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunos consejos para mejorar tu diseño de red:
    
- Incluye una o más capas Dense, con sus funciones de activación apropiadas, antes de la capa de salida.
- Intenta usar una capa de tipo [GlobalAveragePooling](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) en lugar de la capa Flatten. Esta capa calcula una media de todos los valores de píxeles para cada canal, y en algunas ocasiones produce mejores resultados que la capa Flatten.
- ¡Y no olvides todos los consejos del apartado anterior! También aplican aquí.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>

Para mejorar aún más los resultados de tu red, utiliza las siguientes ideas:

- Usa las estrategias PRO del ejercicio anterior.
- Prueba otras redes pre-entrenadas de <a href="https://keras.io/api/applications/">Keras Applications</a>, como Xception o MobileNetV2, o alguna red de las familias EfficientNetV2 o ConvNeXt.
- Usa una estrategia de transfer learning más avanzada, como fine-tuning o una combinación de bottleneck features y fine-tuning. Revisa las diapositivas de clase para saber cómo.
   
Si empleas todos estos trucos, es posible alcanzar más de un 90% de acierto en el conjunto de test.

</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

## Informe final y resumen de resultados

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Escribe en la siguiente celda un pequeño informe con:
    <ul>
        <li>Una tabla de resultados, indicando qué diseños de red has probado y qué resultados en test has obtenido. Puede usar un estilo de tabla como el que se muestra abajo.</li>
        <li>De las estrategias y diseños que has ido probando, ¿qué ha funcionado y qué no?</li>
    </ul>
</font>

***

Ejemplo para la tabla de resultados

|Procesado de imágenes|Diseño de red neuronal|Estrategia de entrenamiento|Acierto en test|
|---------------------|----------------------|---------------------------|---------------|
|Tamaño 32x32, batch size 16|Convolutional(32) + Flatten + Dense(64)|Entrenamiento desde 0|xx%|
|Tamaño 64x64, batch size 32|VGG16 + Flatten + Dense(32)|Bottleneck features|yy%|
|...|...|...|...|